# Singleton pattern
- The singleton design pattern is useful when you need to create only one object or you need some sort of object capable of maintaining a global state for your program.

In [1]:
import urllib.request

In [2]:
class SingletonType(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls._instances.get(cls) is None:
            obj = super(SingletonType, cls).__call__(*args, **kwargs)
            cls._instances[cls] = obj
        return cls._instances[cls]

In [3]:
class URLFetcher(metaclass=SingletonType):
    def __init__(self):
        self.urls = []

    def fetch(self, url):
        req = urllib.request.Request(url)
        with urllib.request.urlopen(req) as response:
            if response.code == 200:
                page_content = response.read()
                with open("content.html", "a") as f:
                    f.write(str(page_content))
                self.urls.append(url)

In [4]:
def main():
    my_urls = [
        "http://python.org",
        "https://planetpython.org/",
        "https://www.djangoproject.com/",
    ]

    print(URLFetcher() is URLFetcher())

    fetcher = URLFetcher()
    for url in my_urls:
        fetcher.fetch(url)

    print(f"Done URLs: {fetcher.urls}")

In [5]:
main()

True
Done URLs: ['http://python.org', 'https://planetpython.org/', 'https://www.djangoproject.com/']


## Prevent race condition

In [6]:
import urllib.request
import threading

In [7]:
class SingletonType(type):
    _instances = {}
    _lock = threading.Lock()  # Ensure thread-safe singleton initialization

    def __call__(cls, *args, **kwargs):
        with cls._lock:
            if cls._instances.get(cls) is None:
                obj = super(SingletonType, cls).__call__(*args, **kwargs)
                cls._instances[cls] = obj
        return cls._instances[cls]

In [8]:
class URLFetcher(metaclass=SingletonType):
    def __init__(self):
        self.urls = []
        self._lock = threading.Lock()  # Protect shared resources

    def fetch(self, url):
        req = urllib.request.Request(url)
        with urllib.request.urlopen(req) as response:
            if response.code == 200:
                page_content = response.read()
                with self._lock:  # Locking before writing to the file and modifying shared resources
                    with open("content.html", "a") as f:
                        f.write(str(page_content))
                    self.urls.append(url)

In [9]:
def main():
    my_urls = [
        "http://python.org",
        "https://planetpython.org/",
        "https://www.djangoproject.com/",
    ]

    print(URLFetcher() is URLFetcher())  # Verify singleton behavior

    fetcher = URLFetcher()

    # Launch threads to fetch URLs concurrently
    threads = []
    for url in my_urls:
        t = threading.Thread(target=fetcher.fetch, args=(url,))
        threads.append(t)
        t.start()

    # Wait for all threads to complete
    for t in threads:
        t.join()

    print(f"Done URLs: {fetcher.urls}")

In [10]:
main()

True
Done URLs: ['http://python.org', 'https://www.djangoproject.com/', 'https://planetpython.org/']
